# Pivots Detection


### Load the data

In [8]:
import pandas as pd
import pandas_ta as ta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats
import datetime
import MetaTrader5 as mt5

if not mt5.initialize(login=40987, server="CabanaCapitals-Demo",password="Mypassword$1234"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

login = 40987
password = 'Mypassword$1234'
server = 'CabanaCapitals-Demo'

mt5.login(login, password, server)

rate = mt5.copy_rates_from('AUDCHF', mt5.TIMEFRAME_M15, datetime.datetime.now(), 24000)
df = pd.DataFrame(rate)

df['date']=pd.to_datetime(df['time'], unit='s')

df.set_index(np.arange(len(df)), inplace = True)
df = df.drop(['spread', 'real_volume', 'tick_volume', 'time'], axis='columns')

# df['RSI'] = ta.rsi(df.close, length=12)
# df['EMA'] = ta.ema(df.close, length=150)


df=df[-5000:]

df.set_index(np.arange(len(df)), inplace = True)

### Trend detection

In [9]:
# EMAsignal = [0]*len(df)
# backcandles = 21

# for row in range(backcandles+15, len(df)):
#     upt = 1
#     dnt = 1
# #     print(row-backcandles)
#     for i in range(row-backcandles, row+1):
#         if max(df.open[i], df.close[i])>=df.EMA[i]:
#             dnt=0
#         if min(df.open[i], df.close[i])<=df.EMA[i]:
#             upt=0
#     if upt==1 and dnt==1:
#         EMAsignal[row]=3
#     elif upt==1:
#         EMAsignal[row]=2
#     elif dnt==1:
#         EMAsignal[row]=1

# df['EMASignal'] = EMAsignal

In [10]:
# def isPivot(candle, window):
#     """
#     function that detects if a candle is a pivot/fractal point
#     args: candle index, window before and after candle to test if pivot
#     returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
#     """
#     if candle-window < 0 or candle+window >= len(df):
#         return 0
    
#     pivotHigh = 1
#     pivotLow = 2
#     for i in range(candle-window, candle+window+1):
#         if df.iloc[candle].low > df.iloc[i].low:
#             pivotLow=0
#         if df.iloc[candle].high < df.iloc[i].high:
#             pivotHigh=0
#     if (pivotHigh and pivotLow):
#         return 3
#     elif pivotHigh:
#         return pivotHigh
#     elif pivotLow:
#         return pivotLow
#     else:
#         return 0

In [11]:
import pandas as pd
import numpy as np
# import plotly.graph_objects as go

# Assuming you have a DataFrame named 'df' containing OHLC data

def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle - window < 0 or candle + window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle - window, candle + window + 1):
        if df.iloc[candle]['low'] > df.iloc[i]['low']:
            pivotLow = 0
        if df.iloc[candle]['high'] < df.iloc[i]['high']:
            pivotHigh = 0
    if pivotHigh and pivotLow:
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

window = 21
df['isPivot'] = df.apply(lambda x: isPivot(x.name, window), axis=1)

def pointpos(x):
    if x['isPivot'] == 2:
        return x['low'] - 1e-3
    elif x['isPivot'] == 1:
        return x['high'] + 1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

dfpl = df[:500]

# Create separate DataFrames for pivot highs and lows
df_pivot_highs = dfpl[dfpl['isPivot'] == 1]

print(df_pivot_highs.iloc[-1])
# df_pivot_lows = dfpl[dfpl['isPivot'] == 2]

# fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
#                 open=dfpl['open'],
#                 high=dfpl['high'],
#                 low=dfpl['low'],
#                 close=dfpl['close'])])

# # Add scatter plots for pivot highs and lows with different colors
# fig.add_scatter(x=df_pivot_highs.index, y=df_pivot_highs['pointpos'], mode="markers",
#                 marker=dict(size=5, color="MediumPurple"),
#                 name="Pivot Highs")
# fig.add_scatter(x=df_pivot_lows.index, y=df_pivot_lows['pointpos'], mode="markers",
#                 marker=dict(size=5, color="Red"),  # You can choose a different color for pivot lows
#                 name="Pivot Lows")

# fig.update_layout(xaxis_rangeslider_visible=False)
# fig.show()


open                    0.58472
high                    0.58477
low                     0.58438
close                   0.58443
date        2023-07-19 03:00:00
isPivot                       1
pointpos                0.58577
Name: 498, dtype: object


In [12]:
window=21
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

In [28]:
def pointpos(x):
    if x['isPivot']==2:
        return x['low']-1e-3
    elif x['isPivot']==1:
        return x['high']+1e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [29]:
dfpl = df[:1000]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [21]:
def detect_structure(candle, backcandles, window):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    localdf = df[candle-backcandles-window:candle-window]  
    highs = localdf[localdf['isPivot'] == 1].high.tail(3).values
    idxhighs = localdf[localdf['isPivot'] == 1].high.tail(3).index
    lows = localdf[localdf['isPivot'] == 2].low.tail(3).values
    idxlows = localdf[localdf['isPivot'] == 2].low.tail(3).index

    pattern_detected = False

    lim1 = 0.005
    lim2 = lim1/3
    if len(highs) == 3 and len(lows) == 3:
        order_condition = (idxlows[0] < idxhighs[0] 
                           < idxlows[1] < idxhighs[1] 
                           < idxlows[2] < idxhighs[2])
        diff_condition = ( 
                            abs(lows[0]-highs[0])>lim1 and 
                            abs(highs[0]-lows[1])>lim2 and
                            abs(highs[1]-lows[1])>lim1 and
                            abs(highs[1]-lows[2])>lim2
                            )
        pattern_1 = (lows[0] < highs[0] and
            lows[1] > lows[0] and lows[1] < highs[0] and
            highs[1] > highs[0] and
            lows[2] > lows[1] and lows[2] < highs[1] and
            highs[2] < highs[1] and highs[2] > lows[2]
            )

        pattern_2 = (lows[0] < highs[0] and
            lows[1] > lows[0] and lows[1] < highs[0] and
            highs[1] > highs[0] and
            lows[2] < lows[1] and
            highs[2] < highs[1] 
            )

        if (order_condition and
            diff_condition and
            (pattern_1 or pattern_2)
        ):
            pattern_detected = True

    if pattern_detected:
        return 1
    else:
        return 0

In [22]:
df['pattern_detected'] = df.index.map(lambda x: detect_structure(x, backcandles=40, window=6))

In [9]:
df[df['pattern_detected']!=0]

,Gmt time,open,high,low,close,volume,RSI,EMA,EMASignal,isPivot,pointpos,pattern_detected
376,26.05.2003 13:00:00.000,1.18084,1.18151,1.18052,1.18126,2.681580e+07,60.322705,1.168325,2,0,NaN,1
377,26.05.2003 14:00:00.000,1.18106,1.18491,1.18059,1.18419,2.690430e+07,73.569795,1.168536,2,0,NaN,1
532,04.06.2003 02:00:00.000,1.17193,1.17320,1.17182,1.17226,2.971090e+07,46.771502,1.175323,1,0,NaN,1
533,04.06.2003 03:00:00.000,1.17243,1.17290,1.17160,1.17221,2.704670e+07,46.578694,1.175282,1,0,NaN,1
534,04.06.2003 04:00:00.000,1.17205,1.17296,1.17107,1.17243,2.887820e+07,47.615243,1.175244,1,0,NaN,1
535,04.06.2003 05:00:00.000,1.17274,1.17283,1.17101,1.17139,2.654940e+07,43.284098,1.175193,1,0,NaN,1
536,04.06.2003 06:00:00.000,1.17094,1.17212,1.17082,1.17097,3.082760e+07,41.616369,1.175137,1,0,NaN,1
537,04.06.2003 07:00:00.000,1.17087,1.17258,1.17066,1.17226,2.712590e+07,48.291884,1.175099,1,0,NaN,1
538,04.06.2003 08:00:00.000,1.17251,1.17463,1.17079,1.17353,2.831090e+07,53.947146,1.175078,1,1,1.17563,1
539,04.06.2003 09:00:00.000,1.17333,1.17406,1.17236,1.17250,2.698430e+07,49.187538,1.175044,1,0,NaN,1
